## general_trash split class update

In [1]:
import os
import json
import shutil

In [2]:
# 원본 JSON 파일 복사
original_json_file = '../../dataset/train.json'
new_json_file = '../../dataset/train_class20.json'
shutil.copyfile(original_json_file, new_json_file)

# 복사된 JSON 파일 로드
with open(new_json_file, 'r') as file:
    data = json.load(file)

# general_trash_dataset 폴더 내의 하위 폴더 탐색
base_folder = '../../general_images_classified_from_original_label'
subfolders = [f.name for f in os.scandir(base_folder) if f.is_dir()]

# 각 하위 폴더를 새로운 카테고리로 추가 (단, 'etc' 폴더는 제외)
for folder in subfolders:
    if folder != 'etc':
        data['categories'].append({
            "id": len(data['categories']),
            "name": folder,
            "supercategory": "General trash"
        })

# annotations 업데이트
for folder in subfolders:
    if folder != 'etc':
        category_id = next(item for item in data['categories'] if item["name"] == folder)["id"]
        image_files = os.listdir(os.path.join(base_folder, folder))

        for file in image_files:
            # 파일명에서 image_id와 id 추출
            parts = file.split('_')
            image_id = int(parts[0])
            annotation_id = int(parts[1].split('.')[0])  # 파일 확장자 제거

            # annotations에서 일치하는 항목 찾기 및 category_id 업데이트
            for annotation in data['annotations']:
                if annotation['image_id'] == image_id and annotation['id'] == annotation_id:
                    annotation['category_id'] = category_id
                    break

# 변경된 JSON 데이터 저장
with open(new_json_file, 'w') as file:
    json.dump(data, file, indent=4)

## Merge category

In [4]:
def merge_categories(json_file, categories_to_merge, final_category_name):
    # JSON 파일 로드
    with open(json_file, 'r') as file:
        data = json.load(file)

    # 합치려는 카테고리의 ID 찾기
    category_ids = [cat['id'] for cat in data['categories'] if cat['name'] in categories_to_merge]

    # 최종 카테고리의 ID (가장 작은 ID)
    final_category_id = min(category_ids)

    # annotations 업데이트
    for annotation in data['annotations']:
        if annotation['category_id'] in category_ids:
            annotation['category_id'] = final_category_id

    # 카테고리 업데이트: 불필요한 카테고리 제거 및 최종 카테고리 이름 변경
    data['categories'] = [cat for cat in data['categories'] if cat['id'] not in category_ids or cat['id'] == final_category_id]
    for cat in data['categories']:
        if cat['id'] == final_category_id:
            cat['name'] = final_category_name

    # 변경된 JSON 데이터 저장
    with open(json_file, 'w') as file:
        json.dump(data, file, indent=4)

# json file path
json_file = '../../dataset/train_class20.json'

# merge하고자하는 category 추가
categories_to_merge = ['box_tape', 'paper_piece']

# merge 후 최종 category name
final_category_name = 'paper_piece'

merge_categories(json_file, categories_to_merge, final_category_name)